In [1]:
#!pip install selenium

In [1]:
# import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains, Chrome
import time
from selenium.webdriver.common.keys import Keys



In [9]:
#get the item list
df_raw_data = pd.read_csv('./useful_data/rice(at_least_50_reviews).csv')

In [10]:
df_raw_data['url']

0     https://www.hktvmall.com/%E5%A4%A7%E8%A1%97%E5...
1     https://www.hktvmall.com/%E5%A4%A7%E8%A1%97%E5...
2     https://www.hktvmall.com/%E5%8D%97%E9%A0%86%EF...
3     https://www.hktvmall.com/%E5%A4%A7%E8%A1%97%E5...
4     https://www.hktvmall.com/%E5%B1%B1%E6%9C%89%E5...
5     https://www.hktvmall.com/%E5%A4%A7%E8%A1%97%E5...
6     https://www.hktvmall.com/%E5%B1%B1%E6%9C%89%E5...
7     https://www.hktvmall.com/%E5%B1%B1%E6%9C%89%E5...
8     https://www.hktvmall.com/%E5%B1%B1%E6%9C%89%E5...
9     https://www.hktvmall.com/%E5%B1%B1%E6%9C%89%E5...
10    https://www.hktvmall.com/%E9%87%91%E6%BA%90%E7...
11    https://www.hktvmall.com/Brilliance-Internatio...
12    https://www.hktvmall.com/Brilliance-Internatio...
13    https://www.hktvmall.com/Brilliance-Internatio...
Name: url, dtype: object

In [16]:
df_item=pd.DataFrame()

df_review=pd.DataFrame()

In [31]:
#review per page

##############################
#####index of the first link##
##############################

##################SET INDEX HERE###################
first_index = 11

index = first_index



test_link = df_raw_data['url'][first_index:]

for link in test_link:
    
    ###################################
    ###Print link index################
    ###################################
    print('Scraping product with link index {}'.format(index))
    index = index + 1

    URL = '{}?scrollTo=reviewTab'.format(link)

    driver = webdriver.Chrome()

    driver.get(URL) #replacing the requests.get step
    

    subhtml = driver.page_source #replacing the html.text

    soup = BeautifulSoup(subhtml, "html.parser")
    #soup.prettify()
    if len(soup.body.find_all(string='此貨品没有評論')):
        print('Product with index {} has no review.'.format(index-1))
        pass
    else:
        dict_item={'ProductID':[],'Name':[],'Brand':[],'Shop':[],'Price':[],'Sales_Number':[],'Package':[],'MadeIn':[],'Description':[],'Image':[],'Review_avg':[],'Review_total':[],'recommendtofdratio':[],'recommendtofd_num':[]}
        dict_review={'ProductID':[],'Username':[],'Review_Date':[],'Comments':[],'Rating':[],'recommendOrNot':[]}

        #df_item_info
        ProductID=soup.find_all('div', {'class' : 'searchKeyword'})[0].text.strip('\n').split('︰')[1]
        dict_item['ProductID'].append(ProductID)
        Name=soup.find_all('h1', {'class' : 'last'})[0].text.split(' - ')
        dict_item['Name'].append(Name[1])
        dict_item['Brand'].append(Name[0])
        dict_item['Shop'].append(soup.find_all('ul', {'class' : 'clearfix'})[0].text.strip('\n'))
        try:
            dict_item['Price'].append(float(soup.find_all('span', {'class' : 'discount'})[0].text.strip('$ ')))
        except:
            dict_item['Price'].append(np.nan)
        try:
            dict_item['Sales_Number'].append(int(soup.find_all('div', {'class' : 'salesNumber-container'})[0].text.strip('\n').strip('已售出 ').strip('+').replace(',','')))
        except:
            dict_item['Sales_Number'].append(np.nan) 
        try:
            dict_item['Package'].append(re.findall('[0-9]+.*',soup.find_all('tr', {'class' : 'productPackingSpec'})[0].text)[0])
        except:
            dict_item['Package'].append(np.nan)
        try:
            dict_item['MadeIn'].append(soup.find_all('tr', {'class' : 'productPackingSpec'})[1].text.strip('\n').strip('產地\n\n\n'))
        except:
            dict_item['MadeIn'].append(np.nan)
        try:
            dict_item['Description'].append(soup.find_all('span', {'class' : 'short-desc'})[0].text)
        except:
            dict_item['Description'].append(np.nan)

        dict_item['Image'].append(re.findall(r'(imagesrc=".*jpg" s)',str(soup.find_all('span', {'class' : 'thumb slick-slide slick-active active'})))[0].strip('imagesrc="').strip('" s'))
        dict_item['Review_avg'].append(soup.find_all('label', {'class' : 'title-text'})[0].text.strip('/5'))
        dict_item['Review_total'].append(re.findall('[0-9]+',soup.find_all('div', {'class' : 'review-count regular-text'})[0].text)[0])
        dict_item['recommendtofdratio'].append(soup.find_all('label', {'class' : 'title-text'})[1].text)
        dict_item['recommendtofd_num'].append(soup.find_all('div', {'class' : 'regular-text'})[2].text)

        df_item_new=pd.DataFrame(dict_item)
        df_item=pd.concat([df_item,df_item_new],axis=0)
        df_item.to_csv('df_item.csv',index=False)

        #df_review
        loop=True
        while loop==True:
            for n in soup.find_all('div', {'class' : 'product-review-wrapper'}):
                dict_review['ProductID'].append(ProductID)
                dict_review['Username'].append(re.search('<span class="review-username">.*</span></a>',str(n))[0].strip('<span class="review-username">').strip('</span></a>'))
                dict_review['Review_Date'].append(re.search('<span class="review-date">.*</span> </td>',str(n))[0].strip('<span class="review-date">').strip('</span> </td>'))
                dict_review['Comments'].append(re.search('<div class="review-title"><span>.*</span>',str(n))[0].strip('<div class="review-title"><span>').strip('</span>'))
                dict_review['Rating'].append(str(n).count('<span class="star"></span>'))
                dict_review['recommendOrNot'].append(str(n).count('<div class="recommendOrNot">'))

            df_review_new=pd.DataFrame(dict_review)
            df_review=pd.concat([df_review,df_review_new],axis=0)
            df_review.to_csv('df_review.csv',index=False)
            #page_num

            try:
                driver.maximize_window()
                button=driver.find_element_by_xpath("//*[@id='reviews']/div[11]/div/a[2]")
                #########################
                ###Added waiting time####
                #########################
                time.sleep(5)
                button.click()
                subhtml = driver.page_source #replacing the html.text
                soup = BeautifulSoup(subhtml, "html.parser")
                #soup.prettify()
            except:
                loop=False
                #########################
                ###Added waiting time####
                #########################
                time.sleep(5)
                driver.close()
print('All item reviews scrapped!')

Scraping product with link index 11
Product with index 11 has no review.
Scraping product with link index 12
Product with index 12 has no review.
Scraping product with link index 13
Product with index 13 has no review.
All items reviews scrapped!


In [12]:
df_item.reset_index(drop=True,inplace=True)
df_item

,ProductID,Name,Brand,Shop,Price,Sales_Number,Package,MadeIn,Description,Image,Review_avg,Review_total,recommendtofdratio,recommendtofd_num
0,J09916,芥花籽油900毫升x3+ 精選絲苗米8公斤,袋鼠米油組合,大街市,165.0,3000,900mlx 3 + 8kg,N/A,芥花籽油900毫升x3+ 精選絲苗米8公斤,//images.hktvmall.com/h0888001/250338/h0888001...,4.7,53,100%,6/6
1,J19163,頂上茉莉香米8公斤+米糠油2公升,瑞茲米油組合,大街市,215.0,2000,8公斤+2升,N/A,頂上茉莉香米8公斤+米糠油2公升,//images.hktvmall.com/h0888001/250695/h0888001...,4.6,55,100%,1/1
2,J18087,純正花生油,刀嘜,南順,128.9,5000,900毫升x4,中國,純正花生油,//images.hktvmall.com/h0888001/a86f5b1becd5083...,4.6,77,66%,4/6
3,C56788,花生油,獅球嘜,大街市,132.9,30000,900亳升 x 4,香港,精選優質新鮮花生，經先進的壓搾工藝提煉而成，保留花生的天然香味，味道香醇。,//images.hktvmall.com/h0888001/65169/h0888001_...,4.6,396,90%,9/10
4,J09916,芥花籽油900毫升x3+ 精選絲苗米8公斤,袋鼠米油組合,大街市,165.0,3000,900mlx 3 + 8kg,N/A,芥花籽油900毫升x3+ 精選絲苗米8公斤,//images.hktvmall.com/h0888001/250338/h0888001...,4.7,53,100%,6/6
5,J19163,頂上茉莉香米8公斤+米糠油2公升,瑞茲米油組合,大街市,215.0,2000,8公斤+2升,N/A,頂上茉莉香米8公斤+米糠油2公升,//images.hktvmall.com/h0888001/250695/h0888001...,4.6,55,100%,1/1
6,J18087,純正花生油,刀嘜,南順,128.9,5000,900毫升x4,中國,純正花生油,//images.hktvmall.com/h0888001/a86f5b1becd5083...,4.6,77,66%,4/6
7,C56788,花生油,獅球嘜,大街市,132.9,30000,900亳升 x 4,香港,精選優質新鮮花生，經先進的壓搾工藝提煉而成，保留花生的天然香味，味道香醇。,//images.hktvmall.com/h0888001/65169/h0888001_...,4.6,396,90%,9/10
8,N99399,泰國100%純正米糠油 1公升,KINGS,山有店,46.0,7000,NaN,NaN,泰國100%純正米糠油 1公升1. 富含谷維素/維生素E米糠油含有豐富的谷維素、維生素E和植...,https://images.hktv-img.com/images/HKTV/623/KR...,4.,68,100%,9/9
9,C56789,獅球嘜粟米油,獅球嘜,大街市,106.9,10000,900亳升 x 4,香港,獅球嘜食油是唯一由香港煉油廠製造的品牌。其粟米油精選上等粟米煉製而成，每份食用份量含8克多元...,//images.hktvmall.com/h0888001/65169/h0888001_...,4.,82,100%,9/9


In [22]:
test_link.iloc[10]

'https://www.hktvmall.com/%E9%87%91%E6%BA%90%E7%B1%B3%E6%A5%AD/s/H1202001/%E8%B6%85%E7%B4%9A%E5%B7%BF%E5%A0%B4/%E8%B6%85%E7%B4%9A%E5%B8%82%E5%A0%B4/%E7%B1%B3-%E9%A3%9F%E6%B2%B9/%E7%B1%B3/%E6%B3%B0%E5%9C%8B%E5%8F%8A%E6%9D%B1%E5%8D%97%E4%BA%9E%E9%A6%99%E7%B1%B3/%E6%BE%B3%E6%B4%B2%E9%9B%99%E7%BE%8A%E7%99%BE%E6%90%AD%E7%B1%B3%E7%8E%8B%E9%9D%9E%E7%9C%9F%E7%A9%BA%E5%8C%85%E8%A3%9D-%E9%9B%99%E7%BE%8A%E7%89%8C%E6%BE%B3%E6%B4%B2%E7%B1%B3%E9%9B%99%E7%BE%8A%E7%B1%B3-/p/H0888001_S_10134098'

In [18]:
df_item

,ProductID,Name,Brand,Shop,Price,Sales_Number,Package,MadeIn,Description,Image,Review_avg,Review_total,recommendtofdratio,recommendtofd_num
0,J09916,芥花籽油900毫升x3+ 精選絲苗米8公斤,袋鼠米油組合,大街市,165.0,3000,900mlx 3 + 8kg,N/A,芥花籽油900毫升x3+ 精選絲苗米8公斤,//images.hktvmall.com/h0888001/250338/h0888001...,4.7,53,100%,6/6
1,J19163,頂上茉莉香米8公斤+米糠油2公升,瑞茲米油組合,大街市,215.0,2000,8公斤+2升,N/A,頂上茉莉香米8公斤+米糠油2公升,//images.hktvmall.com/h0888001/250695/h0888001...,4.6,55,100%,1/1
2,J18087,純正花生油,刀嘜,南順,128.9,5000,900毫升x4,中國,純正花生油,//images.hktvmall.com/h0888001/a86f5b1becd5083...,4.6,77,66%,4/6
3,C56788,花生油,獅球嘜,大街市,132.9,30000,900亳升 x 4,香港,精選優質新鮮花生，經先進的壓搾工藝提煉而成，保留花生的天然香味，味道香醇。,//images.hktvmall.com/h0888001/65169/h0888001_...,4.6,396,90%,9/10
4,J09916,芥花籽油900毫升x3+ 精選絲苗米8公斤,袋鼠米油組合,大街市,165.0,3000,900mlx 3 + 8kg,N/A,芥花籽油900毫升x3+ 精選絲苗米8公斤,//images.hktvmall.com/h0888001/250338/h0888001...,4.7,53,100%,6/6
5,J19163,頂上茉莉香米8公斤+米糠油2公升,瑞茲米油組合,大街市,215.0,2000,8公斤+2升,N/A,頂上茉莉香米8公斤+米糠油2公升,//images.hktvmall.com/h0888001/250695/h0888001...,4.6,55,100%,1/1
6,J18087,純正花生油,刀嘜,南順,128.9,5000,900毫升x4,中國,純正花生油,//images.hktvmall.com/h0888001/a86f5b1becd5083...,4.6,77,66%,4/6
7,C56788,花生油,獅球嘜,大街市,132.9,30000,900亳升 x 4,香港,精選優質新鮮花生，經先進的壓搾工藝提煉而成，保留花生的天然香味，味道香醇。,//images.hktvmall.com/h0888001/65169/h0888001_...,4.6,396,90%,9/10
8,N99399,泰國100%純正米糠油 1公升,KINGS,山有店,46.0,7000,NaN,NaN,泰國100%純正米糠油 1公升1. 富含谷維素/維生素E米糠油含有豐富的谷維素、維生素E和植...,https://images.hktv-img.com/images/HKTV/623/KR...,4.,68,100%,9/9
9,C56789,獅球嘜粟米油,獅球嘜,大街市,106.9,10000,900亳升 x 4,香港,獅球嘜食油是唯一由香港煉油廠製造的品牌。其粟米油精選上等粟米煉製而成，每份食用份量含8克多元...,//images.hktvmall.com/h0888001/65169/h0888001_...,4.,82,100%,9/9


In [19]:
df_review.reset_index(drop=True,inplace=True)
df_review

,ProductID,Username,Review_Date,Comments,Rating,recommendOrNot
0,J09916,Pui Ki Tam,2022-02-21,Che,5,0
1,J09916,Vico I,2022-01-31,有贈品送,5,0
2,J09916,Iris Tso,2022-01-31,用戶已設定此評論內容為非公開,5,0
3,J09916,Eason Baby,2022-01-30,Good,5,0
4,J09916,Jacky Ch,2022-01-27,Good,5,0
...,...,...,...,...,...,...
4187,F80786,Chau E,2019-03-22,120/2包,5,0
4188,F80786,Hui Ling w,2019-03-19,平靚正非常滿意,5,0
4189,F80786,ony ho,2019-03-14,減到$62好抵買，一直食開，但正價八十幾就買櫻城,4,0
4190,F80786,Chau E,2019-03-10,VIP 62蚊仲有95折 抵！,5,0


In [30]:
df_review[df_review['ProductID']=='F80786']

,ProductID,Username,Review_Date,Comments,Rating,recommendOrNot
3642,F80786,M Lau,2022-02-26,非常好👍🏼,5,1
3643,F80786,Abby Sze,2022-02-02,有真空包裝，平過其他超級市場,5,0
3644,F80786,blue kwok,2022-01-25,Ok,3,0
3645,F80786,May LEUNG,2022-01-20,真空包裝👍,5,0
3646,F80786,Wendy Ng,2022-01-07,"Australian rice, soft n moist texture",5,0
...,...,...,...,...,...,...
4187,F80786,Chau E,2019-03-22,120/2包,5,0
4188,F80786,Hui Ling w,2019-03-19,平靚正非常滿意,5,0
4189,F80786,ony ho,2019-03-14,減到$62好抵買，一直食開，但正價八十幾就買櫻城,4,0
4190,F80786,Chau E,2019-03-10,VIP 62蚊仲有95折 抵！,5,0


In [33]:
df_review = df_review.drop_duplicates()

In [35]:
df_review[df_review['ProductID']=='F80786']

,ProductID,Username,Review_Date,Comments,Rating,recommendOrNot
3642,F80786,M Lau,2022-02-26,非常好👍🏼,5,1
3643,F80786,Abby Sze,2022-02-02,有真空包裝，平過其他超級市場,5,0
3644,F80786,blue kwok,2022-01-25,Ok,3,0
3645,F80786,May LEUNG,2022-01-20,真空包裝👍,5,0
3646,F80786,Wendy Ng,2022-01-07,"Australian rice, soft n moist texture",5,0
...,...,...,...,...,...,...
4187,F80786,Chau E,2019-03-22,120/2包,5,0
4188,F80786,Hui Ling w,2019-03-19,平靚正非常滿意,5,0
4189,F80786,ony ho,2019-03-14,減到$62好抵買，一直食開，但正價八十幾就買櫻城,4,0
4190,F80786,Chau E,2019-03-10,VIP 62蚊仲有95折 抵！,5,0
